In [1]:
%pip install -U numpy==1.23.5 transformers==4.26.1
%pip install tensorboard ipywidgets
%pip install pandas
%pip install datasets IPython
%pip install torch torchaudio evaluate tqdm 
%pip install soundfile librosa

     --------------------------------------- 14.7/14.7 MB 16.0 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 19.3 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 KB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 20.1 MB/s eta 0:00:00
     ------------------------------------- 330.1/330.1 KB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 KB 2.2 MB/s eta 0:00:00
     -------------------------------------- 152.8/152.8 KB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 KB 3.3 MB/s eta 0:00:00
     -------------------------------------- 170.9/170.9 KB 5.2 MB/s eta 0:00:00
     -------------------------------------- 121.1/121.1 KB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 61.6/61.6 KB 3.2 MB/s eta 0:00:00
     -------------------------------------- 100.4/100.4 KB 6.0 MB/s eta 0:00:00
     -------------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\rhear\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 5.5/5.5 MB 12.1 MB/s eta 0:00:00
     -------------------------------------- 139.4/139.4 KB 8.1 MB/s eta 0:00:00
     -------------------------------------- 103.9/103.9 KB 6.2 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 KB 2.0 MB/s eta 0:00:00
     -------------------------------------- 226.7/226.7 KB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 18.3 MB/s eta 0:00:00
     -------------------------------------- 413.4/413.4 KB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 18.3 MB/s eta 0:00:00
     -------------------------------------- 215.0/215.0 KB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\rhear\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


     --------------------------------------- 11.6/11.6 MB 19.3 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 KB 3.6 MB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 KB 16.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\rhear\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 536.7/536.7 KB 4.2 MB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 KB 6.6 MB/s eta 0:00:00
     -------------------------------------- 166.4/166.4 KB 3.3 MB/s eta 0:00:00
     --------------------------------------- 24.9/24.9 MB 12.6 MB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 KB 4.0 MB/s eta 0:00:00
     ------------------------------------- 366.0/366.0 KB 11.1 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 KB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 76.9/76.9 KB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 50.7/50.7 KB 1.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\rhear\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os, sys, random, glob
os.environ['TOKENIZERS_PARALLELISM']='false'

import numpy as np
import pandas as pd

import datasets
from datasets import load_dataset, Audio, DatasetDict
from datasets import Audio, Features, ClassLabel

import torch

import transformers
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from transformers import TrainingArguments, Trainer

import evaluate

from IPython.display import display

!git lfs install

In [ ]:
!huggingface-cli login

In [ ]:
labels = ["SAD", 
          "ANGRY",
          "DISGUST",
          "FEAR",
          "HAPPY",
          "NEUTRAL"]


NUM_OF_LABELS = len(labels)

label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

initial_label_update = {"SAD": "SAD", 
                        "ANG": "ANGRY",
                        "DIS": "DISGUST",
                        "FEA": "FEAR",
                        "HAP": "HAPPY",
                        "NEU": "NEUTRAL"}


print(labels)
print(NUM_OF_LABELS)
print(label2id)
print(id2label)

In [ ]:
parent_dir = "/content/drive/MyDrive/Speech Emotion Recognition/data"

dir_path = os.path.join(parent_dir, "*.wav")

files_and_name = glob.glob(dir_path)

metadata = pd.DataFrame(files_and_name, columns=["file_path"])

metadata['file_name'] = metadata['file_path'].apply(lambda x: x.split("/")[-1])

metadata['label'] = metadata['file_path'].apply(lambda x: x.split("/")[-1].split("_")[-2])
metadata['label'] = metadata['label'].replace(initial_label_update)
metadata['label'] = metadata['label'].replace(label2id)

metadata = metadata.drop(columns=["file_path"])

metadata_file_location = os.path.join(parent_dir, "metadata.csv")
metadata.to_csv(metadata_file_location, index=False)

metadata.head()

In [ ]:
audio_data = load_dataset(parent_dir)

audio_data

In [ ]:
audio_data['train'][10]

In [ ]:
audio_data = audio_data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
audio_data = audio_data.shuffle(seed=42)

audio_data_split = audio_data['train'].train_test_split(test_size=0.25)

ds = DatasetDict({
    'train' : audio_data_split['train'],
    'eval' : audio_data_split['test']
})

In [ ]:
print("Training Dataset")
print("Training Dataset Info: ", ds['train'])
print("First Sample in Training Dataset", ds['train'][0])
print("Last Sample in Training Dataset", ds['train'][-1])
print("Unique Values in Label/Class: ", sorted(ds['train'].unique("label")))

print("\n\nEvaluation Dataset")
print("Evaluation Dataset Info: ", ds['eval'])
print("First Sample in Evaluation Dataset", ds['eval'][0])
print("Last Sample in Evaluation Dataset", ds['eval'][-1])
print("Unique Values in Label/Class: ", sorted(ds['eval'].unique("label")))

In [ ]:
for _ in range(5):
    from IPython.display import Audio, display
    rand_idx = random.randint(0, len(ds["train"])-1)
    example = ds["train"][rand_idx]
    audio = example["audio"]
    
    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

In [ ]:
MODEL_CKPT = "facebook/wav2vec2-base"
MODEL_NAME = MODEL_CKPT.split("/")[-1] + "-Speech_Emotion_Recognition"

NUM_OF_EPOCHS = 10
LEARNING_RATE = 3e-5

BATCH_SIZE = 32
STRATEGY = "epoch"

In [ ]:
sampling_rate = ds["train"].features["audio"].sampling_rate
sampling_rate

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_CKPT)

In [ ]:
def preprocess_function(examples):
    '''
    This function prepares the dataset for the transformer
    by applying the feature extractor to it (among other 
    processes).
    '''
    max_duration = 5.0 # seconds
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, 
                               sampling_rate=feature_extractor.sampling_rate, 
                               max_length=int(feature_extractor.sampling_rate * max_duration),
                               truncation=True)
    return inputs

encoded_audio = ds.map(preprocess_function, remove_columns="audio", batched=True)

In [ ]:
def compute_metrics(p):
    '''
    This function calculates & returns the following metrics:
    - accuracy
    - f1 score
    - recall
    - precision
    '''
    import evaluate
    
    accuracy_metric = evaluate.load("accuracy")
    
    accuracy = accuracy_metric.compute(predictions=np.argmax(p.predictions, 
                                                             axis=1), 
                                       references=p.label_ids)['accuracy']
    
    ### ------------------- F1 scores -------------------
    
    f1_score_metric = evaluate.load("f1")
   
    weighted_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, 
                                                                      axis=1), 
                                                references=p.label_ids, 
                                                average='weighted')["f1"]
    
    micro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, 
                                                                   axis=1), 
                                             references=p.label_ids, 
                                             average='micro')['f1']
    
    macro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, 
                                                                   axis=1), 
                                             references=p.label_ids, 
                                             average='macro')["f1"]
    
    ### ------------------- recall -------------------
    
    recall_metric = evaluate.load("recall")
    
    weighted_recall = recall_metric.compute(predictions=np.argmax(p.predictions, 
                                                                  axis=1), 
                                            references=p.label_ids, 
                                            average='weighted')["recall"]
    
    micro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, 
                                                               axis=1), 
                                         references=p.label_ids, 
                                         average='micro')["recall"]
    
    macro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, 
                                                               axis=1), 
                                         references=p.label_ids, 
                                         average='macro')["recall"]
    
    ### ------------------- precision -------------------
    
    precision_metric = evaluate.load("precision")
    
    weighted_precision = precision_metric.compute(predictions=np.argmax(p.predictions, 
                                                                        axis=1), 
                                                  references=p.label_ids, 
                                                  average='weighted')["precision"]
    
    micro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, 
                                                                     axis=1), 
                                               references=p.label_ids, 
                                               average='micro')["precision"]
    
    macro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, 
                                                                     axis=1), 
                                               references=p.label_ids, 
                                               average='macro')["precision"]
    
    return {"accuracy" : accuracy, 
            "Weighted F1" : weighted_f1_score,
            "Micro F1" : micro_f1_score,
            "Macro F1" : macro_f1_score,
            "Weighted Recall" : weighted_recall,
            "Micro Recall" : micro_recall,
            "Macro Recall" : macro_recall,
            "Weighted Precision" : weighted_precision,
            "Micro Precision" : micro_precision,
            "Macro Precision" : macro_precision
            }

In [ ]:
model = AutoModelForAudioClassification.from_pretrained(MODEL_CKPT, 
                                                        num_labels=NUM_OF_LABELS, 
                                                        label2id=label2id,
                                                        id2label= id2label)

In [ ]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy=STRATEGY,
    num_train_epochs=NUM_OF_EPOCHS,
    save_strategy=STRATEGY,
    logging_strategy="steps",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.10,
    gradient_accumulation_steps=4,
    logging_first_step=True,
    report_to="tensorboard",
    hub_private_repo=True,
    push_to_hub=True
)


In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = encoded_audio["train"],
    eval_dataset = encoded_audio["eval"],
    tokenizer = feature_extractor,
    compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub()